In [1]:
import numpy as np

In [2]:
def construct_meshgrid(array):
    dimensions = np.array(array).shape[0]
    meshgrid_parameter_str = ''
    for j in range(dimensions):
        meshgrid_parameter_str += 'array[{}], '.format(j)
  
    meshgrid = eval('np.array(np.meshgrid(' + meshgrid_parameter_str + 'indexing=\'ij\')).T.reshape(-1,dimensions)')

    return meshgrid

In [3]:
def energy(dXY, X_indices, Y_indices, c, cXY, cX, cY):
    XY_combinations = construct_meshgrid(np.array([X_indices, Y_indices]))
    X_combinations = construct_meshgrid(np.array([X_indices, X_indices]))
    Y_combinations = construct_meshgrid(np.array([Y_indices, Y_indices]))
    
    return c*(cXY*np.sum(dXY[XY_combinations[:,0], XY_combinations[:,1]]) \
             -cX*np.sum(dXY[X_combinations[:,0], X_combinations[:,1]]) \
             -cY*np.sum(dXY[Y_combinations[:,0], Y_combinations[:,1]]))


In [4]:
def energy_permutation_test(X, Y, B, alpha):
    n1 = X.shape[0]
    n2 = Y.shape[0]   
    n = n1+n2
    
    X = X.reshape(n1, -1)
    Y = Y.reshape(n2, -1)

    assert(X.shape[1] == Y.shape[1])
    n_dim = X.shape[1]

    c = (n1*n2)/(n1*n2)
    cXY = 2/(n1*n2)
    cX = 1/(n1**2)
    cY = 1/(n2**2)
    
    dXY = np.zeros((n, n))
    for j in range(n_dim):
        XY = np.r_[X[:, j], Y[:, j]]
        dXY += np.subtract.outer(XY, XY)**2
    dXY = np.sqrt(dXY)

    reference_energy = energy(dXY, np.arange(n1), np.arange(n2)+n1, c, cXY, cX, cY)
    
    energies = np.zeros(B)
    for i in range(B):
        permutation = np.random.permutation(n)
        energies[i] = energy(dXY, permutation[:n1], permutation[n1:], c, cXY, cX, cY)
    
    n_larger = (energies[reference_energy > energies]).size
    p = 1-(n_larger/B)
    
    reject = n_larger > (1-alpha)*B
    
    return p, reject


In [5]:
n1 = 20
n2 = 20
ndim = 1

X = np.random.rand(n1, ndim)
Y = np.random.rand(n2, ndim)

B=499
alpha = 0.05
print(energy_permutation_test(X, Y, B, alpha))

(0.40280561122244485, False)


In [6]:
p_values = np.zeros(100)
for i in range(100):
    X = np.random.rand(n1, ndim)
    Y = np.random.rand(n2, ndim)
    p_values[i] = energy_permutation_test(X, Y, B, alpha)[0]

In [7]:
(p_values[p_values <= alpha]).size

5